In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline 

#importing package relating to dashboard
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

#supressing warnings
import warnings
warnings.filterwarnings('ignore')

### Section 1:  Loading in the data required for the interactive dashboard:

In [2]:
forecast_1d = pd.read_csv('_forecast_1d.csv').drop(columns=['Unnamed: 0']).rename(columns={'1d_lstm':'LSTM','1d_arima':'ARIMA'})
forecast_3d = pd.read_csv('_forecast_3d.csv').drop(columns=['Unnamed: 0']).rename(columns={'3d_lstm':'LSTM','3d_arima':'ARIMA'})
forecast_7d = pd.read_csv('_forecast_7d.csv').drop(columns=['Unnamed: 0']).rename(columns={'7d_lstm':'LSTM','7d_arima':'ARIMA'})

In [3]:
tweets_df_ts = pd.read_csv('tweets_time_series.csv')

In [4]:
#updating datetime to a datetime type
forecast_1d['datetime'] = pd.to_datetime(forecast_1d['datetime'])
forecast_3d['datetime'] = pd.to_datetime(forecast_3d['datetime'])
forecast_7d['datetime'] = pd.to_datetime(forecast_7d['datetime'])
tweets_df_ts['datetime'] = pd.to_datetime(tweets_df_ts['datetime'])

In [5]:
forecast_1d.head()

,datetime,LSTM,ARIMA
0,2009-06-25 19:00:00,-0.023032,0.000870
1,2009-06-25 20:00:00,-0.028589,0.000701
2,2009-06-25 21:00:00,-0.031353,0.000731
3,2009-06-25 22:00:00,-0.032367,0.000588
4,2009-06-25 23:00:00,-0.032564,0.000423


In [6]:
tweets_df_ts.head()

,datetime,compound,pos_score,neg_score,neu_score
0,2009-04-07 06:00:00,0.140605,0.236113,0.122964,0.640918
1,2009-04-07 07:00:00,0.160121,0.242813,0.123006,0.634184
2,2009-04-07 08:00:00,0.176024,0.250339,0.106104,0.643562
3,2009-04-07 09:00:00,0.170781,0.241128,0.109694,0.648670
4,2009-04-07 10:00:00,0.184932,0.252483,0.100900,0.646616


In [7]:
forecast_1d.head()

,datetime,LSTM,ARIMA
0,2009-06-25 19:00:00,-0.023032,0.000870
1,2009-06-25 20:00:00,-0.028589,0.000701
2,2009-06-25 21:00:00,-0.031353,0.000731
3,2009-06-25 22:00:00,-0.032367,0.000588
4,2009-06-25 23:00:00,-0.032564,0.000423


### Section 2.  Creating the dashboard using plotly and dash packages:

ref: https://pypi.org/project/plotly-express/
     https://pypi.org/project/dash/#:~:text=Dash%20is%20the%20most%20downloaded,crafted%20%E2%9D%A4%EF%B8%8F%20with%20Dash%20itself).
     
I chose to create this interactive dashboard using dash as I found it easy to get to grips with the code and it has a very large array of possible features

In [8]:
#create the dash app
app = dash.Dash()

#set up the app layout

##-------------------------------LAYOUT-----------------------------------##

app.layout = html.Div(children=[
    html.H1('Twitter Sentiment Forecasting',style={'textAlign': 'center'}),
    ##formatting dropdown for selecting how many days to forecast
    html.P("Select from dropdown how many days ahead to create sentiment forecast:"),
    html.Div(
            [dcc.Dropdown(id='forecast_range',
                          options=[{'label':i, 'value':i} for i in ['1 Day', '3 Days', '7 Days']], 
                          value='1 Day'),html.Br(),],style={'padding': '10px 5px','width': "25%"}
            ),
    
    html.P("Select from dropdown the model to be used for time series forecasting:"),
    html.Div(
            [dcc.Dropdown(id='model_selection',
                          options=[{'label':i, 'value':i} for i in ['LSTM', 'ARIMA', 'Both Models']], 
                          value='LSTM'),html.Br(),],style={'padding': '10px 5px','width': "25%"}
            ),

    #Giving the user the option to only display forecasts
    html.P("Display options:"),
    
    html.Div(
        [dcc.RadioItems(
            ['Including Historical', 'Forecast Only'],
            'Including Historical',
            id='display_options',
            labelStyle={'display': 'inline-block', 'marginTop': '5px'}                
        ),html.Br(),
    ],style={'padding': '10px 5px','width': "25%"}),

       
    #formatting chart
     html.Div(
             [dcc.Graph(id='forecasted_sentiment')]
             ),
         
]) 


##------------------creating functionality of dashboard-----------------##
##creating call back function to plot the forecasted sentiment
@app.callback(
    Output(component_id='forecasted_sentiment', component_property='figure'),
    [Input(component_id ='forecast_range', component_property='value'),
     Input(component_id ='model_selection', component_property='value'), 
     Input(component_id ='display_options', component_property='value')
    ]
    )

def create_forecast_chart(day_filter,model_filter,display):
    #determining which dataframe to use based on selected forecast day
    if day_filter== '1 Day':
        forecasts = forecast_1d
    elif day_filter=='3 Days':
        forecasts = forecast_3d
    elif day_filter=='7 Days':
        forecasts = forecast_7d
    
    if model_filter=='LSTM':
        forecasts = forecasts[['datetime','LSTM']].rename(columns={'LSTM':'forecasts'})
    elif model_filter=='ARIMA':
        forecasts = forecasts[['datetime','ARIMA']].rename(columns={'ARIMA':'forecasts'})
        
    #plotting the chart   
    line_fig = go.Figure()
    x_max = forecasts['datetime'].max() 

    #id "Including Historical" is selected, then include all previous sentiment scores and update the x_min for axis limits
    if display=='Including Historical':
        line_fig.add_trace(go.Scatter(x=tweets_df_ts['datetime'], y=tweets_df_ts['compound'], name='Historical',
                                     mode='lines',marker=dict(color='blue'), showlegend=True))
        
        x_min = tweets_df_ts['datetime'].min()
        x_offset = pd.Timedelta(days=3)
    else:
        x_min = forecasts['datetime'].min()
        x_offset = pd.Timedelta(days=0)
    
    
    #if Both Models is not selected, then display for specific model only
    if model_filter!='Both Models':
        line_fig.add_trace(go.Scatter(x=forecasts['datetime'], y=forecasts['forecasts'], name='Forecast',
                              mode='lines',marker=dict(color='red'), showlegend=True))
        
        line_fig.update_layout(title=f"{day_filter} ahead forecast using {model_filter}; compound score: {forecasts['forecasts'].iloc[-1]}", 
                               xaxis_title="DateTime",yaxis_title="Compound Sentiment",legend_title="Legend")
    else:
        line_fig.add_trace(go.Scatter(x=forecasts['datetime'], y=forecasts['ARIMA'], name='ARIMA',
                                      mode='lines',marker=dict(color='green'), showlegend=True))
        line_fig.add_trace(go.Scatter(x=forecasts['datetime'], y=forecasts['LSTM'], name='LSTM',
                                      mode='lines',marker=dict(color='orange'), showlegend=True))
        
        line_fig.update_layout(title=f"{day_filter} ahead forecast; LSTM forecast: {forecasts['LSTM'].iloc[-1]} and ARIMA forecast:{forecasts['ARIMA'].iloc[-1]}",
                               xaxis_title="DateTime", yaxis_title="Compound Sentiment",legend_title="Legend")

        
    #update layout   
    line_fig.update_layout(xaxis_range=[x_min - x_offset, x_max + x_offset])
    line_fig.update_layout(height=750)
    line_fig.update_layout(title={'xanchor':'center', 'yanchor': 'top', 'y':0.90,'x':0.5,})
    
    line_fig.update_layout(xaxis=dict(rangeslider=dict(visible=True,thickness=0.05)))

    return line_fig


#indicating that I want the dashboard to be opened from another webpage (i.e not within the notebook)
if __name__ == '__main__':
    app.run(jupyter_mode="external")


Dash app running on http://127.0.0.1:8050/


## Section 3: Explaining the dashboard

This dashboard was created with the purpose of providing the user with the ability to clearly and concisely analyse the historical and forecasted sentiment from the tweets dataset.

#### choosing libraries:
- Dash was chosen as the suitable library for building the interactive dashboard as this library excels in terms of UI components, visualization capabilities and due to its popularity, there were plenty of online resources for learning how to get up to speed with it quickly.
- Since dash apps are powered by plotly's open source graphing library, it was decided to create all visualisations on the dashboard using plotly express, since most dash applications use this library for visualing data.


#### Deciding on visualisation using Tufts principles: 
In constructing the visualization displayed in this dashboard, Tuftes principles were constantly referred to in order to make the visualization as accurately portraying the data: 

-  A line plot was chosen as they are effective in displaying variations and patterns over time, with the number of information carrying dimensions not exceeding the number of dimensions in the data. This is determined as the most appropriate way to **"Show the Data"**. 
- Clear, detailed and thorough **labelling** were used to defeat graphical distortion and ambiguity. Important features such as what time represents the Historical data and what represents the Forecasted data are clearly labeled using contrasting colours and a conveniently positioned Legend. Each axis is clearly labeled. The Titles are designed to be dynamic so that they will change as the user changes the parameters for the visualisation. This makes it very clear what the chart is actually displaying. 
- **Non-data ink** is kept at a minimum, however grid lines are incorporated into the visualisation as these add value to the interpretability of the sentiment trend. 


#### dashboard functionality

- The dashboard provides two dropdown features; one for selecting how many days ahead to forecast and one for selecting the model used to produce the forecast. When dealing with multiple choice selections dropdowns are a an appropriate method as they clearly indicate what has been selected to be displayed on the accompanying visualization. The model dropdown also gives the user the choice to select "Both Models" to allow analysis between forecasts.<br>

- Radio item buttons were included to allow the user to only display the forecasted data, giving them a more granular visualization of the forecasts. <br>

- A Scroll bar was added under the visualization to allow a dynamic date range on the line plot<br>
